In [1]:
import numpy as np
from matplotlib import pyplot as plt
from numba import njit
import pandas as pd
import mio
from shapely.geometry import Polygon, Point, LineString
import geopandas as gpd
import random
import pathlib

In [2]:
def bary(ar, P0, P1, P2):
    "take empty array and fill with triangles"
    x0, y0, z0 = P0
    x1, y1, z1 = P1
    x2, y2, z2 = P2
    
    xmin, xmax = min([x0,x1,x2]), max([x0,x1,x2])
    ymin, ymax = min([y0,y1,y2]), max([y0,y1,y2])
    
    det = (y1 -y2)*(x0 - x2) + (x2 - x1)*(y0 - y2)
    if det == 0:
        return
    for y in range(ymin, ymax + 1):
        for x in range(xmin, xmax + 1):
    
            u0 = (y1 - y2)*(x - x2) + (x2 - x1)*(y  - y2)
            l0 = u0 / det
            u1 = (y2 - y0)*(x - x2) + (x0 - x2)*(y  - y2)
            l1 = u1 / det
            l2 = 1 -l0 -l1

            if 0 <= l0 <= 1 and 0 <= l1 <= 1 and 0 <= l2 <= 1:
                z = l0 * z0 + l1 * z1 + l2 * z2
                ar[y, x] = max(ar[y, x], int(z + 0.5))


In [6]:
def make_raster(tri_path, res_tif):
    # read tris
    df = pd.read_csv(tri_path)
    
    # prepare empty array
    xmin = min(df.x0.min(), df.x1.min(), df.x2.min())
    xmax = max(df.x0.max(), df.x1.max(), df.x2.max())
    ymin = min(df.y0.min(), df.y1.min(), df.y2.min())
    ymax = max(df.y0.max(), df.y1.max(), df.y2.max())
    xa = int(xmin)
    ya = int(ymin)
    w = int(xmax - xmin) + 2
    h = int(ymax - ymin) + 2
    xa, ya, w, h
    ar = np.zeros((h, w), dtype=int)
    
    # fill array
    for ind, row in df.iterrows():
        P0 = int(row.x0 - xa), int(row.y0 - ya), int(row.z0 + 0.5)
        P1 = int(row.x1 - xa), int(row.y1 - ya), int(row.z1 + 0.5)
        P2 = int(row.x2 - xa), int(row.y2 - ya), int(row.z2 + 0.5)
        bary(ar, P0, P1, P2)
        
    # make geo tiff
    ra = pd.DataFrame(ar, columns= range(xa, xa+ w), index=range(ya, ya + h))
    ra = ra.sort_index(ascending=False)
    mio.write_raster(ra.astype('uint16'), res_tif)

In [7]:
tris = pathlib.Path(r"C:\loc\_SwissBuildings\tris").glob("*.tri")
tris = list(tris)

In [ ]:
%%time
iall = len(tris)
for i, tri in enumerate(tris):
    mio.show_perc(i, iall, 10)
    res = f'tif/{tri.stem}.tif'
    make_raster(tri, res)

0.0% 0.31% 0.62% 0.93% 1.24% 1.55% 1.86% 2.17% 2.48% 2.79% 3.1% 3.42% 3.73% 4.04% 4.35% 4.66% 4.97% 5.28% 